# Imports

In [12]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from scipy.linalg import svd
import sklearn.cluster as cluster
import plotly.express as px
import plotly.graph_objects as go

np.random.seed(1234)

In [13]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
# mpl.rcParams.update({"axes.grid": True})

# 7.1 Linear Regression

Regression - requires a real-valued outcome

Simple example - model someones weight given their age
More complex - use their age, height, and country of origin

What isn't regression?
Predict if a person is a male or female given data about them? Not because output not real-valued (categorical output)

Simple Regression
- Just one predictor variable, e.g. given gross square footage, what is sale price of house?
- Do a scatter plot of sale price vs. living area

Simple Linear Regression
- Model assumes outcome vs. feature is linear

What is lowess?
- Looks like piecewise-linear
- More difficult to explain though

What is a model?
- Idealized representation of a system
- *All models are wrong, but some are useful*

Linear models
- Models are linear when predictions are linear combinations of variables

### Data Load - California Housing Data Set

In [14]:
# housing = datasets.fetch_california_housing()
# df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
df = pd.read_csv("./data/housing.csv")
display(df)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,

### Scatter with Lowess Trendline

In [15]:
px.scatter(
    df,
    x="GrLivArea",
    y="SalePrice",
    trendline="lowess",
    trendline_color_override="black",
)

# 7.2 Tips Dataset

Row is a table that was served by a waiter. Sex is who paid, not sex of waiter.  
Going to concentrate on total bill

In [16]:
data = sns.load_dataset("tips", cache=False)
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [17]:
px.scatter(
    data,
    x="total_bill",
    y="tip",
    trendline="ols",
    trendline_color_override="black",
)

# 7.3 Creating a Plotly and Scikit-Learn Model

Scikit Process
- Instantiate linear model object
- Fit model
- Use the object to make predictions

### Create Model

In [18]:
features = data[["total_bill"]]
target = data["tip"]
f = linear_model.LinearRegression(fit_intercept=False)
f.fit(features, target)

LinearRegression(fit_intercept=False)

### Predictions

In [19]:
data["prediction"] = f.predict(features)
data.head()

,total_bill,tip,sex,smoker,day,time,size,prediction
0,16.99,1.01,Female,No,Sun,Dinner,2,2.442005
1,10.34,1.66,Male,No,Sun,Dinner,3,1.486188
2,21.01,3.50,Male,No,Sun,Dinner,3,3.019807
3,23.68,3.31,Male,No,Sun,Dinner,2,3.403571
4,24.59,3.61,Female,No,Sun,Dinner,4,3.534367


### Plot

In [20]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=data["total_bill"], y=data["tip"], mode="markers", name="actual")
)
fig.add_trace(
    go.Scatter(
        x=data["total_bill"], y=data["prediction"], mode="lines", name="prediction"
    )
)
fig.update_layout(font_size=20)

### Model Params


In [21]:
display([f.coef_, f.intercept_])

[array([0.1437319]), 0.0]

### Regresion with Plotly

In [22]:
px.scatter(
    data,
    x="total_bill",
    y="tip",
    trendline="ols",
    trendline_color_override="red",
)

# 7.4 Defining Loss Functions

Loss Function
- Numerically computes the badness of a model
- Tells us how wrong it is
- Most common is L2 (squared loss), as in L2 norm

Definition of Loss Function: $$L(y,yhat) = (y - yhat)^2$$

Example:
- y = 5.15
- yhat = 1.04
- loss = (5.15 - 1.04)^2 = 16.89


In [23]:
(5.15 - 1.04) ** 2

16.892100000000003

### MSE

Compute the average of L over entire dataset, mean squared errorr, just take average of all instances of L(y,yhat)

# 7.5 Computing L2


In [24]:
data["l2_loss"] = (data["tip"] - data["prediction"]) ** 2
data.head()

,total_bill,tip,sex,smoker,day,time,size,prediction,l2_loss
0,16.99,1.01,Female,No,Sun,Dinner,2,2.442005,2.050638
1,10.34,1.66,Male,No,Sun,Dinner,3,1.486188,0.030211
2,21.01,3.50,Male,No,Sun,Dinner,3,3.019807,0.230585
3,23.68,3.31,Male,No,Sun,Dinner,2,3.403571,0.008756
4,24.59,3.61,Female,No,Sun,Dinner,4,3.534367,0.005720


In [25]:
data["l2_loss"].mean()

1.1781161154513171

In [26]:
from sklearn.metrics import mean_squared_error

mean_squared_error(data["tip"], data["prediction"])

1.1781161154513171

# 7.6 Optimizing L2

Different choice of slope variable theta --> different predictions --> different L2

In [27]:
mean_squared_error(data["tip"], data["total_bill"] * 0.2)

2.667486278688525

In [28]:
def mse_given_theta(theta: float):
    return mean_squared_error(data["tip"], data["total_bill"] * theta)

In [29]:
mse_given_theta(0.2)

2.667486278688525

In [30]:
thetas = np.linspace(0.1, 0.2, 100)
mses = [mse_given_theta(theta) for theta in thetas]
# display(mses)
fig = px.line(x=thetas, y=mses)
fig.update_layout(xaxis_title="theta", yaxis_title="MSE", font_size=20)

# 7.7 Optimize L2 with Scipy

In [31]:
import scipy.optimize

In [32]:
def g(x):
    return x**3 + x**2 - 3 * x + 2

In [40]:
scipy.optimize.minimize(g, x0=10)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.731646177654308
        x: [ 7.208e-01]
      nit: 10
      jac: [ 4.038e-06]
 hess_inv: [[ 1.576e-01]]
     nfev: 22
     njev: 11

In [34]:
x = np.linspace(-3, 2, 100)
px.line(x=x, y=g(x))

In [36]:
scipy.optimize.minimize(mse_given_theta, x0=0.2).x

array([0.14373189])

In [38]:
# Minimizer can fail!
scipy.optimize.minimize(g, x0=-3)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: -1114853117.349824
        x: [-1.037e+03]
      nit: 1
      jac: [ 3.226e+06]
 hess_inv: [[-3.206e-04]]
     nfev: 236
     njev: 112